# Web Scraping Project: Most Highly-Ranked Films

_Instructions and dataset taken from IBM's [Python Project for Data Engineering](https://www.coursera.org/learn/python-project-for-data-engineering) from Coursera_

# Objectives
- Use the `requests` and `BeautifulSoup` libraries to extract the contents of a web page
- Analyze the `HTML` code of a webpage to find the relevant information
- Extract the relevant information and save it in the required form

# Scenario
Consider that you have been hired by a Multiplex management organization to extract the information of the top 50 movies with the best average rating from the web link shared above.

- The information required is Average Rank, Film, and Year
- Save it to a CSV file
- Load the information to a database under the table with name `Top_50`.

# Prerequisite Steps
## 1. Install required libraries

```bash
python -m pip install -r requirements.txt
```

# Imports and Definitions

In [62]:
# for getting the HTML code of the target page
import requests 

# for exporting data into a database file
import sqlite3

# for data frame compilation and utilities
import pandas as pd 

# for parsing required data from the target page's HTML code
from bs4 import BeautifulSoup

# for deleting the output files, if existing, before the whole scraping process is run
import os


In [63]:
# URL of webpage to scrape
TARGET_URL = "https://web.archive.org/web/20230902185655/https://en.everybodywiki.com/100_Most_Highly-Ranked_Films"

# Filename of output CSV file
TARGET_OUTPUT_CSV_FILE = "top_50_films.csv"

# Filename of output Database file
TARGET_DB_NAME = "Movies.db"

# Table name to use when exporting to Database
TARGET_DB_TABLE_NAME = "Top_50"

# Number of Top 50 Highly-Rated films to scrape
NUM_ENTRIES_TO_SCRAPE = 50

Delete target output files, if existing, to start each run with a clean slate

In [64]:
if os.path.exists(TARGET_OUTPUT_CSV_FILE):
    os.remove(TARGET_OUTPUT_CSV_FILE)

# When re-running, sometimes the database resource is still held by Jupyter Notebook.
# If that should happen, just skip the deletion
try:
    if os.path.exists(TARGET_DB_NAME):
        os.remove(TARGET_DB_NAME)
except:
    pass

# Project Task 1: Load the webpage for scraping
Develop a script that loads the entire web page and parses the resulting text in HTML format. 


In [65]:
# Get the HTML code for the page
html_page = requests.get(TARGET_URL).text

# Parse the HTML code, get all the needed rows (with tag 'tr') from the first table
data = BeautifulSoup(html_page, "html.parser")

# Project Task 2: Analyze the HTML code for relevant information

1. Open the web page in a browser then locate the required table.
2. Right-click on the table and click `Inspect` or something similar, depending on what browser you are using.
![alt text](resources/00_inspect_table.png)
3. This opens the HTML code for the page and takes you directly to the point where the definition of the table begins. Take note of this structure in preparation for webscraping, i.e. what tags to look for for each row.
![alt text](resources/01_html_table.png)

As we can see from the HTML code, the table we are looking for has a `<tbody>` tag inside a `<table>` tag. Using the `find_all()` method of a `bs4` data object, we can get an iterable `ResultSet` for sections containing the tag `table`.

In [66]:
# To figure out the index of the table we need,
# print the 'caption' tag of all tables in the page
tables = data.find_all('table')
print(f"Data type of tables: {type(tables)}")
for i, table in enumerate(tables):
    print(f"Table {i} Caption : {table.find_all('caption')}")

Data type of tables: <class 'bs4.element.ResultSet'>
Table 0 Caption : [<caption>Highest Ranked Films (According to the Top 100 Lists of 5 Sources)
</caption>]
Table 1 Caption : [<caption>Decades with the Most Highly Ranked Films
</caption>]
Table 2 Caption : [<caption>Genres with the Most Highly Ranked Films
</caption>]


From this, we find that the table with caption **"Highest Ranked Films (According to the Top 100 Lists of 5 Sources)"** that we need is the first table (index 0).

Let's store that to a variable `target_table_data`.

In [67]:
# Get the contents of the first table
target_table_data = tables[0]

Next, we see from the browser's 'Inspect' pane that inside the `target_table_data` are the rows denoted by the `<tr>` tag.

Let's take a peek at the first three(3) rows:

In [68]:
target_rows_data = target_table_data.find_all('tr')
for i in range(0,3):
    print(f"Row {i}: {target_rows_data[i]}\n")

Row 0: <tr>
<th>Average Rank</th>
<th>Film</th>
<th>Year</th>
<th>Rotten Tomatoes' Top 100<sup class="reference" id="cite_ref-1"><a href="#cite_note-1">[1]</a></sup></th>
<th>IMDb's Top 250 <sup class="reference" id="cite_ref-2"><a href="#cite_note-2">[2]</a></sup></th>
<th>Empire's Top 100 <sup class="reference" id="cite_ref-3"><a href="#cite_note-3">[3]</a></sup></th>
<th>AFI's Top 100 <sup class="reference" id="cite_ref-4"><a href="#cite_note-4">[4]</a></sup></th>
<th>BFI's Top 100 <sup class="reference" id="cite_ref-5"><a href="#cite_note-5">[5]</a></sup>
</th></tr>

Row 1: <tr>
<td>1</td>
<td>The Godfather</td>
<td>1972</td>
<td>17</td>
<td>2</td>
<td>3</td>
<td>2</td>
<td>21
</td></tr>

Row 2: <tr>
<td>2</td>
<td>Citizen Kane</td>
<td>1941</td>
<td>2</td>
<td>93</td>
<td>40</td>
<td>1</td>
<td>2
</td></tr>



Information gleaned from above:
- The first row contains the column headers, i.e. not all rows contain the data we need
- Each cell containing the data we need is denoted by the `<td>` tag
  - Average Rank: 1st cell in row (index 0)
  - Film Title: 2nd cell in row (index 1)
  - Year: 3rd cell in row (index 2)

# Project Task 3: Scrape the required information
We can now iterate through the rows, taking care to skip rows that do not contain the `<td>` tag, to get the required data.

Note the use of `bs4`'s `contents` field to access the contents of the cell, i.e. excluding the tags.

In [69]:
# Iterate through the rows to find the data from each column (with tag 'td')
count = 0
output_rows_list = []
for row in target_rows_data:
    if count < NUM_ENTRIES_TO_SCRAPE:
        cells = row.find_all('td')
        if len(cells) > 0:
            data_dict = {"Average Rank": int(cells[0].contents[0]),
                        "Film": str(cells[1].contents[0]),
                        "Year": int(cells[2].contents[0])}
            print(f"Found row #{count}: {data_dict}")
            output_rows_list.append(data_dict)
            count = count + 1
    else:
        break

Found row #0: {'Average Rank': 1, 'Film': 'The Godfather', 'Year': 1972}
Found row #1: {'Average Rank': 2, 'Film': 'Citizen Kane', 'Year': 1941}
Found row #2: {'Average Rank': 3, 'Film': 'Casablanca', 'Year': 1942}
Found row #3: {'Average Rank': 4, 'Film': 'The Godfather, Part II', 'Year': 1974}
Found row #4: {'Average Rank': 5, 'Film': "Singin' in the Rain", 'Year': 1952}
Found row #5: {'Average Rank': 6, 'Film': 'Psycho', 'Year': 1960}
Found row #6: {'Average Rank': 7, 'Film': 'Rear Window', 'Year': 1954}
Found row #7: {'Average Rank': 8, 'Film': 'Apocalypse Now', 'Year': 1979}
Found row #8: {'Average Rank': 9, 'Film': '2001: A Space Odyssey', 'Year': 1968}
Found row #9: {'Average Rank': 10, 'Film': 'Seven Samurai', 'Year': 1954}
Found row #10: {'Average Rank': 11, 'Film': 'Vertigo', 'Year': 1958}
Found row #11: {'Average Rank': 12, 'Film': 'Sunset Blvd', 'Year': 1950}
Found row #12: {'Average Rank': 13, 'Film': 'Modern Times', 'Year': 1936}
Found row #13: {'Average Rank': 14, 'Film'

Notice that in the row ranked at 38, at index 37, the film title contains a hyperlink. 

To handle this and other cases like it, we need to add a some special handling:

In [70]:
# Iterate through the rows to find the data from each column (with tag 'td')
count = 0
output_rows_list = []
for row in target_rows_data:
    if count < NUM_ENTRIES_TO_SCRAPE:
        cells = row.find_all('td')
        if len(cells) > 0:

            # For titles with hyperlinks, get the contents of the <a> tag
            # By default, extract film title directly through .contents
            film_title = cells[1].contents[0]
            film_title_hyperlink = cells[1].find('a')
            if film_title_hyperlink != None:
                film_title = film_title_hyperlink.contents[0]

            data_dict = {"Average Rank": int(cells[0].contents[0]),
                        "Film": str(film_title),
                        "Year": int(cells[2].contents[0])}
            print(f"Found row #{count}: {data_dict}")
            output_rows_list.append(data_dict)
            count = count + 1
    else:
        break

Found row #0: {'Average Rank': 1, 'Film': 'The Godfather', 'Year': 1972}
Found row #1: {'Average Rank': 2, 'Film': 'Citizen Kane', 'Year': 1941}
Found row #2: {'Average Rank': 3, 'Film': 'Casablanca', 'Year': 1942}
Found row #3: {'Average Rank': 4, 'Film': 'The Godfather, Part II', 'Year': 1974}
Found row #4: {'Average Rank': 5, 'Film': "Singin' in the Rain", 'Year': 1952}
Found row #5: {'Average Rank': 6, 'Film': 'Psycho', 'Year': 1960}
Found row #6: {'Average Rank': 7, 'Film': 'Rear Window', 'Year': 1954}
Found row #7: {'Average Rank': 8, 'Film': 'Apocalypse Now', 'Year': 1979}
Found row #8: {'Average Rank': 9, 'Film': '2001: A Space Odyssey', 'Year': 1968}
Found row #9: {'Average Rank': 10, 'Film': 'Seven Samurai', 'Year': 1954}
Found row #10: {'Average Rank': 11, 'Film': 'Vertigo', 'Year': 1958}
Found row #11: {'Average Rank': 12, 'Film': 'Sunset Blvd', 'Year': 1950}
Found row #12: {'Average Rank': 13, 'Film': 'Modern Times', 'Year': 1936}
Found row #13: {'Average Rank': 14, 'Film'

Now that we extracted the data required, all that's left to do is to create a Pandas DataFrame out of the list of dicts `output_rows_list`:

In [71]:
output_df = pd.DataFrame.from_dict(output_rows_list)

print(output_df.info(), "\n")
print(output_df)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Average Rank  50 non-null     int64 
 1   Film          50 non-null     object
 2   Year          50 non-null     int64 
dtypes: int64(2), object(1)
memory usage: 1.3+ KB
None 

    Average Rank                                           Film  Year
0              1                                  The Godfather  1972
1              2                                   Citizen Kane  1941
2              3                                     Casablanca  1942
3              4                         The Godfather, Part II  1974
4              5                            Singin' in the Rain  1952
5              6                                         Psycho  1960
6              7                                    Rear Window  1954
7              8                                 Apocalypse Now  1979
8

We have now successfully extracted the Average Rank, Title, and Year data of the Top 50 Most Highly-Ranked Films into a Pandas DataFrame, `output_df`.

## Storing the data
For the last task, we need to load the extracted and transformed data into a CSV file and a Database file.

## Saving the output to a CSV 
The loading to CSV file is easily done using Pandas `to_csv()` method.
The filename for the output CSV file was previously defined as `TARGET_OUTPUT_FILE`.

In [72]:
# Save output to CSV
output_df.to_csv(TARGET_OUTPUT_CSV_FILE)

To verify, we can check the contents of the output CSV file like below:

In [73]:
# Check the output CSV file:
with open(TARGET_OUTPUT_CSV_FILE, 'r') as f:
    print(f.read())

,Average Rank,Film,Year
0,1,The Godfather,1972
1,2,Citizen Kane,1941
2,3,Casablanca,1942
3,4,"The Godfather, Part II",1974
4,5,Singin' in the Rain,1952
5,6,Psycho,1960
6,7,Rear Window,1954
7,8,Apocalypse Now,1979
8,9,2001: A Space Odyssey,1968
9,10,Seven Samurai,1954
10,11,Vertigo,1958
11,12,Sunset Blvd,1950
12,13,Modern Times,1936
13,14,Lawrence of Arabia,1962
14,15,North by Northwest,1959
15,16,Star Wars,1977
16,17,Parasite,2019
17,18,Schindler's List,1993
18,19,Lord of the Rings: The Fellowship of the Ring,2001
19,20,Shawshank Redemption,1994
20,21,It's a Wonderful Life,1946
21,22,Pulp Fiction,1994
22,23,Avengers: Endgame,2019
23,24,City Lights,1931
24,25,One Flew Over the Cuckoo's Nest,1975
25,26,Goodfellas,1990
26,27,Raiders of the Lost Ark,1981
27,28,12 Angry Men,1957
28,29,The Silence of the Lambs,1991
29,30,Taxi Driver,1976
30,31,Saving Private Ryan,1998
31,32,E.T. the Extra Terrestrial,1982
32,33,Alien,1979
33,34,Spider-Man: Into the Spider-verse,2018
34,35,Blade Runner,1982
3

## Storing the output to a CSV
Using the built-in `sqlite3` package and Pandas DataFrame `to_sql()` method, we can export our output dataframe into a database file.

The filename for the output DB filename was previously defined as `TARGET_DB_NAME`, while the target table name was previously defined as `TARGET_DB_TABLE_NAME`.

In [74]:
# Save output to DB
with sqlite3.connect(TARGET_DB_NAME) as conn:
    output_df.to_sql(name=TARGET_DB_TABLE_NAME, con=conn, if_exists='replace', index=False)

To verify, we can check the contents of the DB using Pandas `read_sql()` method like below:

In [75]:
# Check the output DB file by querying:
query_all = 'SELECT *' \
    f' FROM {TARGET_DB_TABLE_NAME};'

query_some = 'SELECT Year, Film' \
    f' FROM {TARGET_DB_TABLE_NAME}' \
    ' WHERE Year BETWEEN 2000 AND 2025' \
    ' ORDER BY Year DESC;'

with sqlite3.connect(TARGET_DB_NAME) as conn:
    print('query_all:\n', pd.read_sql(query_all, con = conn), '\n')
    print('query_some:\n', pd.read_sql(query_some, con = conn), '\n')


query_all:
     Average Rank                                           Film  Year
0              1                                  The Godfather  1972
1              2                                   Citizen Kane  1941
2              3                                     Casablanca  1942
3              4                         The Godfather, Part II  1974
4              5                            Singin' in the Rain  1952
5              6                                         Psycho  1960
6              7                                    Rear Window  1954
7              8                                 Apocalypse Now  1979
8              9                          2001: A Space Odyssey  1968
9             10                                  Seven Samurai  1954
10            11                                        Vertigo  1958
11            12                                    Sunset Blvd  1950
12            13                                   Modern Times  1936
13      

# Acknowledgements
## Course Instructors
- Ramesh Sannareddy
- Joseph Santarcangelo
- Abhishek Gagneja
## Course Offered By
* [IBM Skills Network](https://www.coursera.org/partners/ibm-skills-network)